In [19]:
import py_entitymatching as em
import pandas as pd
import os,sys
import math


In [32]:
## Reading csv tables into pandas dataframe

A=em.read_csv_metadata('Data/csv/yelp_list.csv')
A['ID'] = range(0,len(A))
em.set_key(A,'ID')


B=em.read_csv_metadata('Data/csv/zomato_list.csv')
B['ID'] = range(0,len(B))
em.set_key(B,'ID')

True

In [21]:
def clean_price_range( price_str ):

    avg_price = ''
    
    if not isinstance(price_str,basestring)  and  math.isnan(price_str):
        return price_str

    str2 = price_str.replace('$','')
        
    if '-' in str2:
        num_l = int(str2.split('-')[0])
        num_r = int(str2.split('-')[1])
        num_avg = (num_l+num_r)/2
        avg_price = '$' + str(num_avg)

    else:
        str_word = str2.split()[0].lower()
        num_price  = int(str2.split()[1])
        
        if str_word == 'above':
            num_avg = int(num_price + num_price/2)
            avg_price = '$' + str(num_avg)

        else:
            num_avg = num_price/2
            avg_price = '$' + str(num_avg)
            
    return avg_price

In [22]:
 def clean_num_reviews(review_string) :
      return review_string.split()[0]

In [23]:
def clean_name( name_str):

    name_lower = name_str.lower()

    name_2 = name_lower.split()[-1]
    
    if name_2 == 'restaurant':
        name_lower = name_lower.replace('restaurant','')
        
    return name_lower



In [30]:
def clean_address( address ):
    addr_2 = address.replace('\"','')
    addr_3 = addr_2.replace(',','')
    if "Street" in addr_3:
        addr_3 = addr_3.replace('Street','St')
    if "Boulevard" in addr_3:
        addr_3 = addr_3.replace('Boulevard','Blvd')
        
    return addr_3

In [33]:
########## DATA CLEANING ################

##Converting A's price range to an absolute number
A['price_range'] = A['price_range'].apply( clean_price_range )


A['name'] = A['name'].apply( clean_name )

B['name'] = B['name'].apply( clean_name )
B['address'] = B['address'].apply( clean_address )
B['number_of_reviews'] = B['number_of_reviews'].apply( clean_num_reviews )



















In [34]:
#To find missing values 
len(A.ID) - A.count()

number_of_reviews     0
price_range          44
ratingValue           0
name                  0
address               0
ID                    0
dtype: int64

In [35]:
len(B.ID) - B.count()

number_of_reviews    0
price_range          0
ratingValue          0
name                 0
address              0
ID                   0
dtype: int64

In [36]:
block_f = em.get_features_for_blocking(A,B)


In [37]:
block_f




,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,price_range_price_range_lev_dist,price_range,price_range,None,None,lev_dist,<function price_range_price_range_lev_dist at 0x7fb393efa398>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,price_range_price_range_lev_sim,price_range,price_range,None,None,lev_sim,<function price_range_price_range_lev_sim at 0x7fb393efa230>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,price_range_price_range_jar,price_range,price_range,None,None,jaro,<function price_range_price_range_jar at 0x7fb393ff99b0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,price_range_price_range_jwn,price_range,price_range,None,None,jaro_winkler,<function price_range_price_range_jwn at 0x7fb393ff9b18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,price_range_price_range_exm,price_range,price_range,None,None,exact_match,<function price_range_price_range_exm at 0x7fb3c846af50>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,price_range_price_range_jac_qgm_3_qgm_3,price_range,price_range,qgm_3,qgm_3,jaccard,<function price_range_price_range_jac_qgm_3_qgm_3 at 0x7fb391c34410>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,name_name_jac_qgm_3_qgm_3,name,name,qgm_3,qgm_3,jaccard,<function name_name_jac_qgm_3_qgm_3 at 0x7fb391c34a28>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,name_name_cos_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,cosine,<function name_name_cos_dlm_dc0_dlm_dc0 at 0x7fb391c34c08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,name_name_jac_dlm_dc0_dlm_dc0,name,name,dlm_dc0,dlm_dc0,jaccard,<function name_name_jac_dlm_dc0_dlm_dc0 at 0x7fb391c34b18>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,name_name_mel,name,name,None,None,monge_elkan,<function name_name_mel at 0x7fb391c349b0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [38]:
##Rule based on restaurant name similarity, 0.65 score is derived from debugging blocking output
rb1 = em.RuleBasedBlocker()
rule1 = 'name_name_lev_sim(ltuple,rtuple) < 0.65'
rb1.add_rule(rule1, block_f )

##Rule based on address similarity
rb2 = em.RuleBasedBlocker()
rule2 = 'address_address_jac_qgm_3_qgm_3(ltuple,rtuple) < 0.7'
rb2.add_rule(rule2,block_f)


'_rule_0'

In [39]:
## To block based on zipcode equivalence

def zipcode_match(x, y):
    # x, y will be of type pandas series
    
    # get address attribute
    x_address = x['address']
    y_address = y['address']
    
    # get the zipcode
    x_split, y_split = x_address.split(), y_address.split()
    x_zipcode = x_split[len(x_split) - 1]
    y_zipcode = y_split[len(y_split) - 1]
    
    # check if the zipcode match
    if x_zipcode != y_zipcode:
        return True
    else:
        return False

In [40]:
# Instantiate blackbox blocker
bb = em.BlackBoxBlocker()

# Set the black box function
bb.set_black_box_function(zipcode_match)

In [41]:
## Blocking Pipeline- First block based on zip code then block based on address similarity##

C1 = bb.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],n_jobs=-1)

C2 = rb2.block_candset(C1,n_jobs=-1)

###Direct name blocker###
C3 = rb1.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],n_jobs=-1)

## Direct Address blocker not enabled##
#C2 = rb2.block_tables(A,B,l_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'],r_output_attrs=['ID','name','address','ratingValue','price_range','number_of_reviews'])

C4 = em.combine_blocker_outputs_via_union([C2, C3])


0%                          100%
[##############################] | ETA: 00:03:34 | ETA: 00:03:31 | ETA: 00:03:26 | ETA: 00:03:13 | ETA: 00:03:01 | ETA: 00:02:52 | ETA: 00:02:45 | ETA: 00:02:38 | ETA: 00:02:31 | ETA: 00:02:24 | ETA: 00:02:17 | ETA: 00:02:09 | ETA: 00:02:02 | ETA: 00:01:55 | ETA: 00:01:48 | ETA: 00:01:40 | ETA: 00:01:33 | ETA: 00:01:26 | ETA: 00:01:18 | ETA: 00:01:11 | ETA: 00:01:04 | ETA: 00:00:57 | ETA: 00:00:50 | ETA: 00:00:42 | ETA: 00:00:35 | ETA: 00:00:28 | ETA: 00:00:21 | ETA: 00:00:14 | ETA: 00:00:06 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:03:25
0%                          100%
[                              ]0%                          100%
[##############################] | ETA: 00:00:16 | ETA: 00:00:14 | ETA: 00:00:12 | ETA: 00:00:12 | ETA: 00:00:12 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA:

In [42]:
## Debugging blocker to make sure true positives are not left out in candidate set
dbg = em.debug_blocker(C4,A,B,output_size=400)


In [43]:
dbg


,_id,similarity,ltable_ID,rtable_ID,ltable_number_of_reviews,ltable_price_range,ltable_ratingValue,ltable_name,ltable_address,rtable_number_of_reviews,rtable_price_range,rtable_ratingValue,rtable_name,rtable_address
0,0,0.538462,2365,1834,16,$5,4.0,beto’s mexican,3518 E 7th St Austin TX 78702,19,$40,3.2,angie's mexican,1307 E 7th St Austin TX 78702
1,1,0.538462,2433,563,367,$45,4.0,place pigalle,81 Pike St Seattle WA 98101,43,$110,3.8,maximilien,Pike Place Market 81 Pike St Seattle WA 98101
2,2,0.538462,1391,1606,1396,$5,4.5,torchy’s tacos,1311 S 1st St Austin TX 78704,17,$40,3.7,mellizoz tacos,1503 S 1St St Austin TX 78704
3,3,0.529412,1927,1107,225,$20,3.5,maudie’s milagro,3801 N Capital Of Texas Hwy Austin TX 78746,11,$65,4.0,360 uno,3801 N Capital of Texas Hwy Suite G100 Austin TX 78746
4,4,0.529412,1717,1192,1013,$5,4.0,pho y #1,1660 E Capitol Expy San Jose CA 95121,2,$70,3.8,pho y #1 noodle house,1660 East Capitol Expressway San Jose CA 95121
5,5,0.500000,1652,1483,266,$20,4.0,bocaphê,222 Lafayette St New York NY 10012,17,$70,4.1,cuba,222 Thompson St New York NY 10012
6,6,0.500000,1032,727,423,$20,4.0,55 south,55 S 1st St San Jose CA 95113,4,$70,3.4,nemea,96 South 1st St San Jose CA 95113
7,7,0.500000,2143,159,212,$20,4.0,balzem,202 Mott St New York NY 10012,212,$140,4.5,balthazar,80 Spring St New York NY 10012
8,8,0.500000,1219,1251,26,$20,4.5,treehaus mima,470 W 42nd St New York NY 10036,26,$15,4.2,schmackary's,362 W 45th St New York NY 10036
9,9,0.500000,2997,18,821,$20,3.0,the cheesecake factory,925 Blossom Hill Road San Jose CA 95123,8,$80,4.3,bj's,Westfield Oakridge 925 Blossom Hill Road San Jose CA 95123


In [44]:
##Writing final candidate set to CSV file
em.to_csv_metadata(C4,'./C4_output.csv')

True

In [29]:
C4


,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_ratingValue,ltable_price_range,ltable_number_of_reviews,rtable_name,rtable_address,rtable_ratingValue,rtable_price_range,rtable_number_of_reviews
0,0,0,29,dee dee,1906 E Cesar Chavez St Austin TX 78702,5.0,$20,125,la barbecue,1906 E Cesar Chavez St Austin TX 78702,4.0,$40,38
1,1,0,138,dee dee,1906 E Cesar Chavez St Austin TX 78702,5.0,$20,125,juan in a million,2300 E Cesar Chavez St Austin TX 78702,4.0,$25,66
2,2,0,1105,dee dee,1906 E Cesar Chavez St Austin TX 78702,5.0,$20,125,bufalina,1519 E Cesar Chavez St Austin TX 78702,3.4,$50,9
3,3,0,1415,dee dee,1906 E Cesar Chavez St Austin TX 78702,5.0,$20,125,counter culture,2337 E Cesar Chavez St Austin TX 78702,3.6,$30,12
4,4,0,2747,dee dee,1906 E Cesar Chavez St Austin TX 78702,5.0,$20,125,las cazuelas mexican,1701 E Cesar Chavez St Austin TX 78702,3.3,$45,7
5,5,0,3164,dee dee,1906 E Cesar Chavez St Austin TX 78702,5.0,$20,125,mongers market and kitchen,2401 E Cesar Chavez St Austin TX 78702,3.4,$75,2
6,6,1,67,canlis,2576 Aurora Ave N Seattle WA 98109,4.0,$91,1041,canlis,2576 Aurora Avenue North Seattle WA 98109,4.1,$150,87
7,7,2,2538,gopoké,625 S King St Seattle WA 98104,4.5,$20,146,quality athletics,121 S King St Seattle WA 98104,3.1,$50,20
8,8,3,2686,thursday kitchen,424 E 9th St New York NY 10009,4.5,$20,266,tray kitchen,4012 Leary Way NW Seattle WA 98107,3.1,$50,9
9,9,4,1028,simple,109 Eldridge St New York NY 10002,4.5,$20,159,vanessa's dumpling house,118A Eldridge St New York NY 10002,4.0,$15,31
